**Pipeline:**  

import spreadsheet -> preprocess matches (columns in toydata) ->  
create elo class -> add players for teams-> init team dicts -> bymatch: expectation, gameOver, update dicts/rating ->  
massage final results into output format -> save output -> websitey stuff

In [11]:
#imports
import openpyxl
import pandas as pd
import requests
import json
from io import BytesIO
from operator import itemgetter

In [4]:
# Read in toy data spreadsheet tabs as separate dataframes in a list, season_data
spreadsheetId = "1mH_HUNLF_hj2lekOxKMOKM7RiTko4HvjdJryC5M69bM" # Please set your Spreadsheet ID.
url = "https://docs.google.com/spreadsheets/export?exportFormat=xlsx&id=" + spreadsheetId
res = requests.get(url)
data = BytesIO(res.content)
xlsx = openpyxl.load_workbook(filename=data)
season_data = []
for name in xlsx.sheetnames:
    new_df = pd.read_excel(data, sheet_name=name)
    new_df.name = name
    season_data.append(new_df)

# Need to remove some rows in the Match Records, filter via rows where matches are played
matches = season_data[0]
matches = matches[matches['Played?'] == True]
season_data[0] = matches

In [5]:
matches.head()

,Team 1,Team 2,Week,Played?,Map 1,Winner,Map 2,Winner.1,Map 3,Winner.2,Map 4,Winner.3,Map 5,Winner.4,Match Winner
0,A,B,1.0,True,Lijiang Tower,A,Dorado,A,King's Row,B,Hanamura,A,NaN,NaN,A
1,C,D,1.0,True,Lijiang Tower,D,Havana,D,Eichenwalde,C,Temple of Anubis,C,Busan,D,D
2,E,F,1.0,True,Lijiang Tower,E,Dorado,E,King's Row,E,NaN,NaN,NaN,NaN,E
3,G,H,1.0,True,Lijiang Tower,H,Havana,G,King's Row,H,Temple of Anubis,H,NaN,NaN,H
4,I,J,1.0,True,Lijiang Tower,I,Dorado,J,Eichenwalde,J,Hanamura,J,NaN,NaN,J


In [8]:
# Define ELO model
K = 125 # Update scaler
Rm = 3200 # mean elo initialization
delta = 500

class Team:
    def __init__(self, name, Rm=3200):
        self.currentRating = Rm
        self.opponentHist = []
        self.ratingHist = []
        self.currentRanking = 0
        
class Elo:
    def __init__(self,Rm = 3200, k=125, g=1, delta=500):
        self.ratingDict = {}
        self.teamsDict = {}
        self.k = k
        self.g = g
        self.rankings = {}

    def addPlayer(self,name, Rm = 3200):
        self.ratingDict[name] = Rm # update code by deleting this and subsuming into teams dict
        self.teamsDict[name] = Team(name, Rm)

    def gameOver(self, winner, loser):
        # take single game, chop up number of maps & winner of each
        # combine all updates at once
        #   expectation =  expected score | number of maps & team elos
        #   diff = total map wins (+.5 for draws) - expectation
        result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser]) # compute the expected result
        
        # redundancy
        self.ratingDict[winner] = self.ratingDict[winner] + (self.k*self.g)*(1 - result)  # update elo rating based on error
        self.ratingDict[loser] 	= self.ratingDict[loser] + (self.k*self.g)*(0 - (1 -result))
        
        self.teamsDict[winner].currentRating =  self.ratingDict[winner] + (self.k*self.g)*(1 - result)
        self.teamsDict[loser].currentRating = self.ratingDict[loser] + (self.k*self.g)*(0 - (1 -result))  
                                                                            
        # update the histories
        self.teamsDict[winner].opponentHist.append(loser)
        self.teamsDict[winner].ratingHist.append(self.teamsDict[winner].currentRating)
        self.teamsDict[loser].opponentHist.append(winner)
        self.teamsDict[loser].ratingHist.append(self.teamsDict[loser].currentRating)
                                                                            

    def expectResult(self, p1, p2):
        exp = (p2-p1)/delta
        return 1/((10.0**(exp))+1)
    
    def update_rankings(self):
        r = 1
        for ranking in self.power_rankings():
            team = ranking[1][0]
            self.teamsDict[team].currentRanking = r
            r += 1
            
    def power_rankings(self):
        self.rankings = sorted(self.ratingDict.items(), key=itemgetter(1), reverse=True)
        power = []
        for i, x in enumerate(self.rankings):
            power.append((i + 1, x))
        return power

# Testing Section

**MACRO TEST**


In [9]:
# initialize the elo model
GWL = Elo()

# initialize teams
teams = ['A','B','C','D','E','F','G','H','I','J']
for team in teams:
    GWL.addPlayer(team)

for i, game in matches.iterrows():
    # Identify winner and loser
    winner = game['Match Winner']
    if game['Team 1'] == winner:
        loser = game['Team 2']
    else:
         loser = game['Team 1']
    # update elo
    GWL.gameOver(winner,loser)
    
#vars(GWL.teamsDict['A'])
GWL.update_rankings()
GWL.teamsDict['A'].currentRanking

4

In [30]:
# NEXT STEP: MASSAGE OUTPUT INTO FORMAT WE WANT (json)
TeamsJSON = {}
for team in GWL.teamsDict:
    teamJSON = json.dumps(GWL.teamsDict[team].__dict__)
    TeamsJSON[team] = teamJSON
TeamsJSON

{'A': '{"currentRating": 3340.813330989171, "opponentHist": ["B", "G", "D", "E", "C"], "ratingHist": [3325.0, 3352.4837500492786, 3402.1698951821977, 3229.888665325035, 3340.813330989171], "currentRanking": 4}',
 'B': '{"currentRating": 3474.526922938871, "opponentHist": ["A", "C", "H", "D", "E"], "ratingHist": [3075.0, 3262.5, 3355.3219798424416, 3367.6447399704994, 3474.526922938871], "currentRanking": 1}',
 'C': '{"currentRating": 2934.381475848924, "opponentHist": ["D", "B", "E", "I", "A"], "ratingHist": [3075.0, 3012.5, 3014.936731661989, 2920.7016974167414, 2934.381475848924], "currentRanking": 10}',
 'D': '{"currentRating": 3219.548407452145, "opponentHist": ["C", "E", "A", "B", "J"], "ratingHist": [3325.0, 3137.5, 3105.3219798424416, 3062.677239871942, 3219.548407452145], "currentRanking": 7}',
 'E': '{"currentRating": 3265.6286860651862, "opponentHist": ["F", "D", "C", "A", "B"], "ratingHist": [3325.0, 3387.5, 3385.063268338011, 3479.9738539473888, 3265.6286860651862], "curren

In [34]:
# To  Access the nested dicitonary requires the use of json.loads() and the team name (key) to be loaded
json.loads(TeamsJSON['A'])

{'currentRating': 3340.813330989171,
 'opponentHist': ['B', 'G', 'D', 'E', 'C'],
 'ratingHist': [3325.0,
  3352.4837500492786,
  3402.1698951821977,
  3229.888665325035,
  3340.813330989171],
 'currentRanking': 4}

In [ ]:
# THE NEXT GOAL WILL BE TO OUTPUT A GRAPH OF THE HISTORIES, A Table W/ rankings and change of rankings between weeks, and confusion matrix?
